In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import requests
import pandas as pd
import json
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
API_key = os.getenv("weather_api")

In [5]:
city = "London"
url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric"

In [6]:
# an example API call to London
weather_ex = requests.get(url)
weather_ex.status_code

200

In [7]:
weather_ex.json()["list"]

[{'dt': 1655305200,
  'main': {'temp': 26.9,
   'feels_like': 26.52,
   'temp_min': 25.81,
   'temp_max': 26.9,
   'pressure': 1019,
   'sea_level': 1019,
   'grnd_level': 1015,
   'humidity': 34,
   'temp_kf': 1.09},
  'weather': [{'id': 804,
    'main': 'Clouds',
    'description': 'overcast clouds',
    'icon': '04d'}],
  'clouds': {'all': 100},
  'wind': {'speed': 1.69, 'deg': 164, 'gust': 3.54},
  'visibility': 10000,
  'pop': 0,
  'sys': {'pod': 'd'},
  'dt_txt': '2022-06-15 15:00:00'},
 {'dt': 1655316000,
  'main': {'temp': 25.69,
   'feels_like': 25.28,
   'temp_min': 23.26,
   'temp_max': 25.69,
   'pressure': 1019,
   'sea_level': 1019,
   'grnd_level': 1015,
   'humidity': 37,
   'temp_kf': 2.43},
  'weather': [{'id': 804,
    'main': 'Clouds',
    'description': 'overcast clouds',
    'icon': '04d'}],
  'clouds': {'all': 95},
  'wind': {'speed': 1.97, 'deg': 51, 'gust': 3.13},
  'visibility': 10000,
  'pop': 0,
  'sys': {'pod': 'd'},
  'dt_txt': '2022-06-15 18:00:00'},
 {'d

In [8]:
weatherex_df = pd.json_normalize(weather_ex.json()["list"])
weatherex_df.head()

,dt,weather,visibility,pop,dt_txt,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,main.sea_level,main.grnd_level,main.humidity,main.temp_kf,clouds.all,wind.speed,wind.deg,wind.gust,sys.pod,rain.3h
0,1655305200,"[{'id': 804, 'main': 'Clouds', 'description': ...",10000,0.0,2022-06-15 15:00:00,26.90,26.52,25.81,26.90,1019,1019,1015,34,1.09,100,1.69,164,3.54,d,NaN
1,1655316000,"[{'id': 804, 'main': 'Clouds', 'description': ...",10000,0.0,2022-06-15 18:00:00,25.69,25.28,23.26,25.69,1019,1019,1015,37,2.43,95,1.97,51,3.13,d,NaN
2,1655326800,"[{'id': 803, 'main': 'Clouds', 'description': ...",10000,0.0,2022-06-15 21:00:00,20.75,20.21,17.68,20.75,1019,1019,1016,51,3.07,63,2.43,84,4.05,n,NaN
3,1655337600,"[{'id': 802, 'main': 'Clouds', 'description': ...",10000,0.0,2022-06-16 00:00:00,15.96,15.39,15.96,15.96,1020,1020,1017,68,0.00,29,1.23,74,1.98,n,NaN
4,1655348400,"[{'id': 802, 'main': 'Clouds', 'description': ...",10000,0.0,2022-06-16 03:00:00,14.88,14.30,14.88,14.88,1021,1021,1018,72,0.00,40,1.05,10,1.08,n,NaN


In [9]:
weatherex_df = weatherex_df[["dt", "pop", "dt_txt", "main.temp", "main.humidity", "clouds.all", "wind.speed", "wind.gust"]]
weatherex_df.head()

,dt,pop,dt_txt,main.temp,main.humidity,clouds.all,wind.speed,wind.gust
0,1655305200,0.0,2022-06-15 15:00:00,26.90,34,100,1.69,3.54
1,1655316000,0.0,2022-06-15 18:00:00,25.69,37,95,1.97,3.13
2,1655326800,0.0,2022-06-15 21:00:00,20.75,51,63,2.43,4.05
3,1655337600,0.0,2022-06-16 00:00:00,15.96,68,29,1.23,1.98
4,1655348400,0.0,2022-06-16 03:00:00,14.88,72,40,1.05,1.08


In [10]:
# read the city dataframe to get the names of all cities we want weather data for
city_data = pd.read_csv("city_data.csv")
city_data.drop(columns = ["Unnamed: 0"], inplace = True)
city_data.head()

,city_id,city,country,country_code,region,elevation,city_latitude,city_longitude,population
0,Q64,Berlin,Germany,DE,Berlin,NaN,52.516667,13.383333,3664088
1,Q1055,Hamburg,Germany,DE,Hamburg,NaN,53.550000,10.000000,1852478
2,Q1726,Munich,Germany,DE,Bavaria,519.0,48.134520,11.571000,1488202
3,Q365,Cologne,Germany,DE,North Rhine-Westphalia,52.0,50.942222,6.957778,1083498
4,Q1794,Frankfurt am Main,Germany,DE,Hesse,112.0,50.113611,8.679722,764104


In [12]:
# convert the city column to a list so it can be used as an input of the function call
cities = city_data["city"].to_list()

In [13]:
# function that makes a call to the weather API and concatenate weather data from all cities of interest into
# one dataframe
def get_weather_data(cities):
    weather_list = []
    url = f"http://api.openweathermap.org/data/2.5/forecast?q=Boston&appid={API_key}&units=metric"
    test = requests.get(url)
    if test.status_code >= 200 and test.status_code <= 299:
        for city in cities:
            url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric"
            weather = requests.get(url)
            weather_df = pd.json_normalize(weather.json()["list"])
            weather_df["city"] = city
            weather_list.append(weather_df)
    else:
        return -1
    weather_combined = pd.concat(weather_list, ignore_index = True)
    return weather_combined

In [14]:
weather_data = get_weather_data(cities)
weather_data

,dt,weather,visibility,pop,dt_txt,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,...,main.grnd_level,main.humidity,main.temp_kf,clouds.all,wind.speed,wind.deg,wind.gust,sys.pod,rain.3h,city
0,1655305200,"[{'id': 800, 'main': 'Clear', 'description': '...",10000,0.00,2022-06-15 15:00:00,26.10,26.10,25.81,26.10,1006,...,1013,35,0.29,0,1.15,55,2.37,d,NaN,Berlin
1,1655316000,"[{'id': 801, 'main': 'Clouds', 'description': ...",10000,0.00,2022-06-15 18:00:00,25.27,24.79,23.61,25.27,1010,...,1012,36,1.66,19,2.90,71,3.11,d,NaN,Berlin
2,1655326800,"[{'id': 801, 'main': 'Clouds', 'description': ...",10000,0.00,2022-06-15 21:00:00,20.79,20.20,18.13,20.79,1013,...,1012,49,2.66,23,3.78,93,8.92,n,NaN,Berlin
3,1655337600,"[{'id': 802, 'main': 'Clouds', 'description': ...",10000,0.00,2022-06-16 00:00:00,15.84,15.15,15.84,15.84,1017,...,1012,64,0.00,31,2.66,86,5.99,n,NaN,Berlin
4,1655348400,"[{'id': 804, 'main': 'Clouds', 'description': ...",10000,0.00,2022-06-16 03:00:00,15.35,14.66,15.35,15.35,1016,...,1011,66,0.00,100,1.84,71,2.54,d,NaN,Berlin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,1655683200,"[{'id': 500, 'main': 'Rain', 'description': 'l...",10000,0.91,2022-06-20 00:00:00,19.06,19.37,19.06,19.06,1017,...,951,90,0.00,80,1.69,198,1.57,n,0.37,Innsbruck
596,1655694000,"[{'id': 500, 'main': 'Rain', 'description': 'l...",10000,0.81,2022-06-20 03:00:00,17.89,18.16,17.89,17.89,1017,...,951,93,0.00,100,1.66,204,1.52,n,2.13,Innsbruck
597,1655704800,"[{'id': 500, 'main': 'Rain', 'description': 'l...",10000,0.85,2022-06-20 06:00:00,21.82,22.25,21.82,21.82,1016,...,951,84,0.00,91,0.50,228,0.93,d,1.19,Innsbruck
598,1655715600,"[{'id': 501, 'main': 'Rain', 'description': 'm...",10000,0.79,2022-06-20 09:00:00,24.81,25.33,24.81,24.81,1015,...,951,76,0.00,76,1.61,22,2.61,d,3.32,Innsbruck


In [15]:
# set dataframe to keep only the columns of interest
weather_data = weather_data[["pop", "dt_txt", "main.temp", "main.feels_like", "main.humidity", "clouds.all", 
                             "wind.speed", "wind.gust", "city"]]

In [16]:
# rename columns with more intuitive names and get rid of dots to avoid problems in MySQL
weather_data.rename(columns = {"pop": "precip_prob", 
                               "dt_txt": "forecast_time", 
                               "main.temp": "temperature",
                               "main.feels_like": "feels_like",
                               "main.humidity": "humidity", 
                               "clouds.all": "cloudiness", 
                               "wind.speed": "wind_speed", 
                               "wind.gust": "wind_gust",}, 
                    inplace = True)

In [17]:
# add the city_id column to the weather dataframe which will be used as the foreign key in MySQL
weather_data = weather_data.merge(city_data[["city_id", "city"]], how = "left", on = "city")

In [18]:
weather_data.sample(10)

,precip_prob,forecast_time,temperature,feels_like,humidity,cloudiness,wind_speed,wind_gust,city,city_id
245,0.00,2022-06-16 06:00:00,16.87,16.31,65,46,2.99,5.01,Düsseldorf,Q1718
256,0.00,2022-06-17 15:00:00,28.04,27.22,32,100,2.21,2.93,Düsseldorf,Q1718
300,0.00,2022-06-18 03:00:00,18.59,18.12,62,3,2.20,5.22,Dortmund,Q1295
345,0.00,2022-06-18 18:00:00,31.71,31.10,35,2,1.19,2.25,Essen,Q2066
551,0.00,2022-06-19 12:00:00,30.99,32.04,47,0,2.40,2.16,Salzburg,Q34713
566,0.33,2022-06-16 09:00:00,23.84,23.87,61,61,1.42,1.93,Innsbruck,Q1735
233,0.00,2022-06-19 18:00:00,29.74,30.25,47,0,2.59,5.71,Stuttgart,Q1022
524,0.00,2022-06-16 03:00:00,15.42,15.16,82,6,1.90,1.75,Salzburg,Q34713
54,0.00,2022-06-17 09:00:00,20.55,19.97,50,75,2.35,3.16,Hamburg,Q1055
221,0.00,2022-06-18 06:00:00,19.97,19.54,58,17,1.48,2.45,Stuttgart,Q1022


In [19]:
weather_data.dtypes

precip_prob      float64
forecast_time     object
temperature      float64
feels_like       float64
humidity           int64
cloudiness         int64
wind_speed       float64
wind_gust        float64
city              object
city_id           object
dtype: object

In [21]:
weather_data["forecast_time"] = pd.to_datetime(weather_data["forecast_time"])

In [22]:
weather_data.to_csv("weather_data.csv")